### Day 21: Keypad Conundrum

Link: https://adventofcode.com/2024/day/21

To solve this problem, we need to:

1. Generate all possible shortest instructions for the numeric keypad.
2. Generate all possible shortest instructions for the first directional keypad using the instructions generated in the previous step. 
3. Find the shortest instruction for the second directional keypad using the instructions generated in the previous step.

To find the shortest instructions for a given input, we perform a breadth-first search, collecting all paths that lead to the target position and stopping when new paths are longer than the shortest path found so far. We also need to avoid visiting positions marked with "*" as specified in the problem.

In [17]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
from collections import deque


codes: list[str] = []


for line in lines:
    codes.append(line.strip())


numpad = [
    ["7", "8", "9"],
    ["4", "5", "6"],
    ["1", "2", "3"],
    ["*", "0", "A"],
]
arrow_pad = [
    ["*", "^", "A"],
    ["<", "v", ">"],
]
directions = {
    "^": (-1, 0),  # Up
    ">": (0, 1),  # Right
    "v": (1, 0),  # Down
    "<": (0, -1),  # Left
}


def is_within_grid(row: int, column: int, grid: list[list[str]]) -> bool:
    return 0 <= row < len(grid) and 0 <= column < len(grid[0])


def generate_keypad_instructions(
    keypad: list[list[str]],
    start_row: int,
    start_column: int,
    target_instruction: str,
    generate_all: bool = False,
) -> list[str]:
    instructions = [""]
    current_row, current_column = start_row, start_column

    for digit in target_instruction:
        to_visit = deque([(current_row, current_column, "", set())])
        new_instructions: list[str] = []

        while to_visit:
            row, column, instruction, visited = to_visit.popleft()

            if (row, column) in visited:
                continue

            if new_instructions and len(instruction) >= len(new_instructions[0]):
                break

            if keypad[row][column] == digit:
                new_instructions.append(instruction + "A")
                current_row, current_column = row, column

                if generate_all:
                    continue

                break

            for direction, (add_row, add_column) in directions.items():
                new_row, new_column = row + add_row, column + add_column

                if (
                    is_within_grid(new_row, new_column, keypad)
                    and keypad[new_row][new_column] != "*"
                ):
                    to_visit.append(
                        (
                            new_row,
                            new_column,
                            instruction + direction,
                            visited | {(row, column)},
                        )
                    )

        instructions = [
            instruction + new_instruction
            for instruction in instructions
            for new_instruction in new_instructions
        ]

    return instructions


def generate_numpad_instructions(code: str, generate_all: bool = False) -> list[str]:
    return generate_keypad_instructions(numpad, 3, 2, code, generate_all)


def generate_arrow_pad_instructions(
    instruction: str, generate_all: bool = False
) -> list[str]:
    return generate_keypad_instructions(arrow_pad, 0, 2, instruction, generate_all)


complexity_sum = 0


for code in codes:
    numpad_instructions = generate_numpad_instructions(code, True)
    new_instructions: list[str] = []

    for instruction in numpad_instructions:
        keypad_instructions = generate_arrow_pad_instructions(instruction, True)
        new_instructions.extend(keypad_instructions)

    shortest_instruction_length = float("inf")

    for instruction in new_instructions:
        keypad_instructions = generate_arrow_pad_instructions(instruction)
        keypad_instruction = keypad_instructions[0]

        if len(keypad_instruction) < shortest_instruction_length:
            shortest_instruction_length = len(keypad_instruction)

    complexity = shortest_instruction_length * int(code[:-1])
    complexity_sum += complexity


print(complexity_sum)